# Python and other Software

In addition to everything that Python can do natively, Python can also interact with other programs. It can do so in a number of ways:

1. Calling programs using system calls - either directly like clicking on an icon, or from a console (including using batch files and calling in command files that work with the program).
2. Using User Interface Automation to interact with the Graphical User Interface
3. Interact with a program's Application Programming Interfaces (API)
4. Internal Scripting tool - if the program incorporates Python in this way

![](Resources\Images\Diagrams_2.png)

## Reference
Python Interactions with other programs https://stackoverflow.com/questions/14288177/interact-with-other-programs-using-python

## Running External Programs as a Process
Python has a library called [subprocess](https://docs.python.org/3/library/subprocess.html) for running external processes. Note that it replaces two older libraries (`os.system` and `os.spawn` which appear in many online examples).

We will focus on two commands: `run` and `Popen`.

They can be used to call programs with parameters, which may include command files.

In [ ]:
# We can open a program on a computer - note that Python waits for the program to finish
import subprocess as sp
programName = "notepad.exe"
sp.run([programName])

In [ ]:
# We can also use it to open an existing file by passing a parameter to the program call
programName = "notepad.exe"
fileName = "file.txt"
proc1 = sp.run([programName, fileName])
print(proc1)

In [ ]:
# We can use a different approach to run a program in parallel - subprocess.Popen()
programName = "notepad.exe"
fileName = "file.txt"
proc2 = sp.Popen([programName, fileName])
print(proc2)

In [ ]:
# We can check to see what the state of the process is using the poll() command
pid2 = proc2.pid
print(pid2)  # Print the Process ID
poll2 = proc2.poll()
print(poll2)
# Answer is 'None' if process is still running, '0' if process has ended.

## Driving External Programs through the User Interface (UI)
This is the same thing that AutoIT and AutoHotKeys do. There are a few Python libraries that can be used to do this (including one that is a wrapper for AutoIT - [pyAutoIT](https://github.com/jacexh/pyautoit)). The best one is probably [PyWinAuto](https://pywinauto.github.io). The following example is of using PyWinAuto to drive NotePad through the User Interface. Note the calls to "Menus".

PyWinAuto can identify and interact with controls (menu, menu items, buttons, tree views etc) so that it is not necessary to know where the items are.

The following examples show how you can use pyautowin to drive Notepad and Oasys AdSec.

### Notepad driven through the GUI using pyWinAuto
The following code drives Notepad by interacting with the graphical objects that are drawn to the screen by the Graphical User Interface (GUI).

In this case pyWinAuto is used to:
1. start the application using the `start` command, then 
2. select the Help Menu using the `menu_select` command to bring up the "About" window,
3. close it again using the description of the dialog (`AboutNotepad`) with the name of the button (`OK`), and
4. type some text into the main window. 


In [ ]:
from pywinauto.application import Application
# Run a target application
app = Application().start("notepad.exe")
# Select a menu item
app.UntitledNotepad.menu_select("Help->About Notepad")
# Click on a button
app.AboutNotepad.OK.click()
# Type a text string
app.UntitledNotepad.Edit.type_keys("pywinauto Works!", with_spaces = True)
app.UntitledNotepad.DrawOutline()

### Oasys AdSec driven through the GUI using pyWinAuto
The following lines start up Oasys AdSec and runs an N/M interaction and exports the results to a text file.

Starting up a program can be done using the following format, where `start_command` is a string that will cause your program to start. 
```python
app = Application(backend="win32").start(start_command) # for older programs
app = Application(backend="uia").start(start_command) # for newer programs
```

In the case of AdSec, `start_command` should be the path to the AdSec executable and probably also the name of the file that you wish to open:
```python
start_command = r'"C:\Program Files\Oasys\AdSec 8.4\AdSec.exe" R400x200_C45.adtxt'
```
1. Note the  `r` at the beginning to make sure that the text string is not interpreted by Python (which would mess with the `\` delimeters. The single quotes `'` are used to enclose the command string so as to differentiate them from the double quotes `"` which are part of the command.
2. Note the double quotes round the path for the AdSec executable - this is to make sure that the spaces in the folder names don't break up the path.

Automating Oasys AdSec is not all that easy, because of the way that the window title changes. Nevertheless, this is quite possible.

In [ ]:
# Start up AdSec
#adsec_file = r'Test_sec_1.adtxt'
adsec_file = r'R400x200_C45.adtxt'
start_command = r'"C:\Program Files\Oasys\AdSec 8.4\AdSec.exe" ' + adsec_file
adsec_app = Application(backend="uia").start(start_command) # for newer programs

In [ ]:
# Create a variable that points to the AdSec Window
#adsec_win = app.window(title_re=adsec_file + ".*") 
adsec_win = adsec_app.window(title_re=adsec_file + " - Oasys Adsec") 
print(adsec_win)
if adsec_win.exists():
    print('AdSec window exists')
    print('AdSec:')
    print(adsec_win)
    print(adsec_win.wrapper_object())
else:
    print('E R R O R - AdSec window does not exist')

![](Resources\Images\AdSec_01.png)
Identifying windows, dialogs and controls (buttons, comboboxes etc) can be challenging. However, there is a command that will help identify windows, dialogs and controls - `print_control_identifiers()`. 

In [ ]:
# There is a command that will help identify windows, dialogs and controls - 'print_control_identifiers()'
adsec_win.print_control_identifiers()

Once the items have been identified (for example `Axial_load_Moment_interaction_chartButton`), they can be addressed in a number of ways using the main window variable (in this case `adsec_win`):
* Using dictionary look-up format: `adsec_win['Axial load/Moment interaction chartButton']`
* Using object format (by converting spaces and non-alphabetic characters to underscores: `adsec_win.Axial_load_Moment_interaction_chartButton`

In [ ]:
# Highlight the button for carrying out an NM analysis
adsec_win.Axial_load_Moment_interaction_chartButton.draw_outline()
#adsec_win['Axial load/Moment interaction chartButton'].draw_outline()

In [ ]:
# Click on the button to carry out an NM analysis
adsec_win['Axial load/Moment interaction chartButton'].click()

In [ ]:
# Click OK 
adsec_win['OKButton'].click()

In [ ]:
# Maximise the chart window - not actually required - and causes problems here
adsec_win['MaximizeButton'].click()

![](Resources\Images\AdSec_02.png)
Note that the title of the AdSec window has changed...

In [ ]:
# note that the name of the AdSec window has changed...
print('The AdSec window variable works' if adsec_win.exists() else 'The AdSec window variable does not work')

In [ ]:
# We need to get the new name
max_win_title = '[' + adsec_file + ' : N/M]'
max_win = adsec_app[max_win_title]
print('Chart window found' if max_win.exists() else 'Chart window not found')

In [ ]:
# Here we need to do a right-button click, so we need a new desktop function
from pywinauto import Desktop
dsk = Desktop(backend='uia')

if max_win.exists():
    max_win.set_focus().click_input(button='right')
    dsk.Context['Export Curve...'].click_input()

In [ ]:
# Check to see if SaveAs dialog has appeared.
# If so highlight box for file name and use 'type_keys' to input the name for the export file

print('SaveAs dialog found' if adsec_app.Save_AsDialog.File_name_Edit.exists() else 'SaveAs dialog not found')
if adsec_app.Save_AsDialog.File_name_Edit.exists():
    adsec_app.Save_AsDialog.File_name_ComboBox.draw_outline()
    adsec_app.Save_AsDialog.File_name_ComboBox.type_keys('curve_datapoints.csv')

In [ ]:
# Click on the Save Button
adsec_app.Save_AsDialog.SaveButton.click_input()

In [ ]:
# exit from AdSec using 'Alt-F4'
max_win.type_keys('%{F4}')

In [ ]:
# Click on NoButton so as not to save
adsec_app.OpenDialog.NoButton.click_input()

### References

More information on using the Graphical User Interface can be found in the following references:
* [PyWinAuto website - Introduction](https://pywinauto.github.io/)
* [Documentation](https://pywinauto.readthedocs.io/en/latest/contents.html)
* [pyAutoIT Repository](https://github.com/jacexh/pyautoit)
* [Topics tagged with 'PyWinAuto' on Stack Overflow](https://stackoverflow.com/questions/tagged/pywinauto)


## Driving External Programs through the Application Programming Interface (API)
We have already seen an example of this - namely XLwings. 
![XLwings](Resources\Images\xlwings_landingpage.png)

Other examples include:
* [RevitPythonShell](https://github.com/architecture-building-systems/revitpythonshell) - note that this is not the same as Dynamo - it is a more direct interaction.
* [AutoCAD including Civil3D](https://pyautocad.readthedocs.io/en/stable/) - this can be used to automate both AutoCAD and other programs that share the same API.
* [ArcGIS](https://developers.arcgis.com/python/) - this is carried out using a library called [arcpy](http://resources.arcgis.com/en/help/main/10.1/index.html#//000v00000001000000#GUID-4EC90E5F-F497-4FC0-99FB-7703ED4C8F77) ([Python Training is available from ESRI](https://www.esri.com/training/catalog/57630436851d31e02a43f13c/python-for-everyone/))
* [GSApy](https://gitlab.arup.com/david.dekoning/gsapy) - a Python wrapper for GSA
* [Bentley](https://www.bentley.com/en/software-developers/developing-engineering-applications/design-and-analytical-modeling-sdks) - Software Development Kit (SDK)
* [Plaxis](https://www.plaxis.com/news/software-update/plaxis-remote-scripting/) - for remote scripting using HTTP REST API.

An increasing number of Oasys programs include APIs and many can be driven from Python. The following are examples using GSA. 

### Driving GSA through the Application Programming Interface (API)
This example uses the Common Object Model (COM) Application Programming Interface (API) frovided by GSA. This is provided as an example of what you can do with such interfaces. However, if you really want to drive GSA, I would recommend using the GSApy library which is more user-friendly.

In [ ]:
# If you have GSA, then you may want to test out this
from comtypes.client import CreateObject
from comtypes import CLSCTX_LOCAL_SERVER 
from os.path import abspath, exists

infile = "Seismic Example (GSA 8_7) 02.gwb"
print("{} - input file exists".format(infile)) if exists(infile) else print ('{} - input file is not in folder'.format(infile))

# Create GSA COM object
GsaObj = CreateObject("Gsa.ComAuto",CLSCTX_LOCAL_SERVER)
GSA_PID = GsaObj.ProcessID()
print(GSA_PID)

# Send commands to the GSA COM object
status = GsaObj.Open(abspath(infile))
node = GsaObj.GwaCommand("HIGHEST, NODE")

print("Highest node is :", node)

In [ ]:
GsaObj.NodeCoor(164)
nds = list(range(1,1000))
nodes = GsaObj.Nodes(nds)
#GsaObj.SaveAs("delme_now.gwb")
len(nodes[0])
#print(GsaObj.NodeCoor(nodes[2]))
#ignore = [print(nd) for nd in nodes]

In [ ]:
GSA_PID = GsaObj.ProcessID()
print("GSA Process ID:  ", GSA_PID, "\n")
print("GSA Component Versions:")
#print(GsaObj.VersionString())

In [ ]:
# Close down GSA
GsaObj.Close()
GsaObj = None

### GSA and GSApy
The following functions from `gsapy` are demonstrated for reference. This library is already installed with the Arup Anaconda installation. For more information take a look at the [README documentation](https://gitlab.arup.com/david.dekoning/gsapy/blob/master/README.md).

In [ ]:
from gsapy import GSA
infile = "Seismic Example (GSA 8_7) 02.gwb"

In [ ]:
model = GSA(infile,version="9.0")

In [ ]:
print(model.version())

In [ ]:
# This creates lists and dictionaries for working with nodes and elements
node_list = model.get_nodes()
node_dict = {n.index:n for n in node_list}    # Dictionary comprehension
elem_list = model.get_elements()
elem_dict = {e.index:e for e in elem_list}    # Dictionary comprehension
print("Number of nodes:", len(node_list), "\nNumber of elements:", len(elem_list))

In [ ]:
# The node list allows you to cycle through all nodes
# Note that item 2 is not necessarily node 2
n_2 = node_list[2]
print('Node number:', n_2.index, '\nNode coords:', n_2.coords)

In [ ]:
# The node dictionary allows you to look up nodes and node properties
n2 = node_dict.get(2)
print('Node number:', n2.index,   '\nNode coords:', n2.coords)

In [ ]:
e2 = elem_list[2]
print('Element type:', e2.type, '\nProperty:', e2.prop, '\nTopo:', e2.topo) 

In [ ]:
model.close()
del model

## Calling Dynamic Linked Libraries (DLLs)
> "A dynamic-link library (DLL) is an executable file that acts as a shared library of functions and resources. Dynamic linking enables an executable to call functions or use resources stored in a separate file. These functions and resources can be compiled and deployed separately from the executables that use them. The operating system can load the DLL into the executable's memory space when the executable is loaded, or on demand at runtime. DLLs also make it easy to share functions and resources across executables. Multiple applications can access the contents of a single copy of a DLL in memory at the same time." (Source: Microsoft)

Python can draw on the functionality provided in the form of Dynamic Linked Libraries (DLLs). These are typically written using C, C++, C#.NET F#.NET or VB.NET and may be referenced using the `pythonnet` library in CPython, or using `clr` in IronPython.

If the pythonnet library is not already installed, you can install it from a conda repository. Run the cell below to open up a command prompt, then type  `conda install -c pythonnet pythonnet` to install the library.

In [ ]:
import os   # opening up a command window console
os.system(r"start /B start cmd.exe @cmd /k C:\ProgramData\Anaconda3\Scripts\activate.bat C:\ProgramData\Anaconda3")

The advantage of compiling code to DLLs is that they can be re-used in many locations. Note that the best way of using DLLs is to code them in VB.NET, C#.NET or F#.NET and then distribute as DLLs. They can be used in Python scripts by using the `clr` library to reference them in. Note that the `clr` function in CPython and IronPython are different, but they do the same thing.

In [ ]:
import clr
from pathlib import Path
dllDir = Path('.')
dllPath = Path(dllDir / 'doubleMe.dll')  #-- this defines the DLL as being in that directory

print(str(dllPath.resolve()))
#  this uses the relative path above (but resolved to the full path) to call the DLL
ref = clr.AddReference (str(dllPath.resolve()))
print(ref)

The 'doubleMe' DLL was actually written in IronPython and contains only two functions and has no classes or modules.

In [ ]:
from doubleMe import double_me, triple_me
print('double_me(7) =', double_me(7))
print('triple_me(7) =', triple_me(7))

## Python as Internal Scripting Program
Some programs have integrated Python, which means that you are able to write scripts using Python. This is not the same as running the program through the API, since the instance of Python is internal.

Examples include:
* [Rhino / Grasshopper](https://developer.rhino3d.com/guides/rhinopython/)
* [Dynamo](http://primer.dynamobim.org/en/10_Custom-Nodes/10-4_Python.html)
* [ArcGIS Pro](http://pro.arcgis.com/en/pro-app/arcpy/get-started/installing-python-for-arcgis-pro.htm) - comes with Python 3 embedded
* ABAQUS
* Blender
* Paraview
* ...[and many others](https://en.wikipedia.org/wiki/List_of_Python_software#Embedded_as_a_scripting_language)...

## Process Utilities - listing active processes
The `psutil` utility is quite powerful and can provide extensive information on the function of the computer (even including fan speeds and CPU temperatures!). Extensive ([documentation is provided](https://psutil.readthedocs.io/en/latest/)).

In [ ]:
# Listing all active processes - can be modified to only return instances of GSA - remove the '#' in the commands below.
import psutil
format1 = "{:5s} {:5s} {:5s}   {:36s} {:s}"
format2 = "{:5d} {:5.2f} {:5.2f}   {:36s} {:s}"
print(format1.format("PID", "%CPU", "%MEM", "NAME", "PATH"))
for proc in psutil.process_iter(attrs=['pid', 'name', 'cwd', 'cpu_percent', 'memory_percent']):
    pi = proc.info
    try:
        print(format2.format(pi["pid"], pi["cpu_percent"], pi["memory_percent"], 
                             pi["name"], pi["cwd"])) # if pi["name"] == "GSA.exe" else None
    except:
        print(format2.format(pi["pid"], pi["cpu_percent"], pi["memory_percent"], 
                             pi["name"], "<Not Available>")) # if pi["name"] == "GSA.exe" else None
        